## Vorbereitung

:t Just Nothing

=> Maybe (Maybe a)

`const 0 x` ist eine funktion, die für alle Werte einfach 0 liefert

In [ ]:
-- Nothing :: Maybe a
-- Just :: a -> Maybe a
-- vordefiniert als `maybe`
foldMaybe :: b -> (a -> b) -> Maybe a -> b
foldMaybe n f Nothing = n
foldMaybe n f (Just a) = f a

In [ ]:
-- Left :: a -> Either a b
-- Right :: b -> Either a b
foldEither :: (a -> c) -> (b -> c) -> Either a b -> c
foldEither l _ (Left x) = l x
foldEither _ r (Right x) = r x

```
map (foldEither id (\b -> if b then 1 else 0)) [Left 42, Right True, Left False]
=> [42,1,0]

((+1) (*2)) 7
=> 15
```
d.h es wird immer die hintere funktion zuerst ausgeführt

In [ ]:
(.) :: (b -> c) -> (a -> b) -> a -> c
f . g = \x -> f (g x)
-- (.) f g x = f (g x)
-- (f . g) x = f (g x)

```
:t ($)
=> (a -> b) -> a -> b

const 42 ([2,3] ++ [4,5]) == const 42 $ [2,3] ++ [4,5]
=> 42
```

## Abstrakte Datentypen
### Array
```
putIndex :: Int -> a -> Array a -> Array a
getIndex :: Int -> Array a -> Maybe a ODER a
```

In [2]:
type Array a = Int -> a

emptyArray :: Array a
emptyArray y = error ("Access to undefined index: " ++ show y)

getIndex :: Int -> Array a -> a
getIndex i arr = arr i

putIndex :: Int -> a -> Array a -> Array a
-- putIndex i x arr = \j -> if j == i then x else arr j
putIndex i x arr j = if j == i then x else arr j

removeIndex :: Int -> Array a -> Array a 
-- das wird nicht removed, sondern an der stelle einfach nur erweitert und dann vorne dran halt steht, dass das nicht mehr da ist. aber im hintergrund gibts das schon noch
removeIndex i arr = \j -> if  j == i then emptyArray j else arr j

-- TEST
let a = emptyArray
let a1 = putIndex 42 True a
-- let a1 = \j -> if j == 42 then True else emptyArray j
let a2 = putIndex 73 True a1
-- let a2 = \j -> if j == 73 then True else (\j -> if j == 42 then True else emptyArray j)
let a3 = putIndex 55 False a2
-- getIndex 55 a3


"hallo"

### Liste von Funktionen
das ist eine Liste von funktionen, mit den Funktionen [(n -> n+1)...]

*Kurzer Exkurs:* "Hello" !! 3 = "l"

In [25]:

fun :: Int -> [Int -> Int]
fun 0 = []
-- fun n = fun (n-1) ++ [\m -> m+n]
fun n = fun (n-1) ++ [(n+)]

(fun 10 !! 3) 38
-- partiell appliziert

42

### Typklassen
`Num, Ord, Eq`

In [30]:
data Tree = Leaf Int | Tree :-: Tree

-- induktiv über den Datentyp
instance Eq Tree where
    Leaf x == Leaf y        = x == y
    l1 :-: r1 == l2 :-: r2  = l1 == l2 && r1 == r2
    _ == _                  = False

das ist ganz witzig, hier mache ich jetzt ne neudefinition mit deriving, das hätte ich auch mit `Show` machen können, dann hätte ich mir den Kram bei der `prettyPrintJSON` funktion sparen können

In [38]:
-- hier ist es jetzt nich tmehr so einfach, weil ich nicht direkt x mit y vergleichen kann. ich weiss ja nicht, was das für werte sind

data T a = L a | T a :+: T a
-- deriving (Eq) wäre dasselbe gewesen

-- wenn ein eq kontext für a existiert, dann definiere ich jetzt einen für a 
instance Eq a => Eq (T a) where    
    L x == L y              = x == y
    l1 :+: r1 == l2 :+: r2  = l1 == l2 && r1 == r2
    _ == _                  = False

instance Ord a => Ord (T a) where
    compare (L _) (_ :+: _) = LT
    compare (_ :+: _) (L _) = GT
    compare (L x) (L y)     = compare x y
    compare (l1 :+: r1) (l2 :+: r2) = case compare l1 l2 of
                                        EQ -> compare r1 r2
                                        other -> other -- das ist jetzt einfach das ergebnis von compare l1 l2
                                        -- LT -> LT
                                        -- GT -> GT


angenommen ich habe eine funtion die induktiv definiert ist als:
``` 
f x = 42 
g x = g (f x)
```
und dann den wert: 
`f (g 42)`

da gibts dann 2 Möglichkeiten

outermost               | innermost     |
|--|--|
42                      | f(g (f 42))   |
42                      | f(g 42)       |
42                      | f(g (f 42))   |
42                      | ...           |
haskell                 | andere Sprachen|

- das ist krass, weil ich jetzt mit unendlichen Listen arbeiten kann

```
ones :: [Int]
ones = 1 : ones
```
jetzt kann ich `ones !! 1000000` machen
(aber das kann der nicht im speicher anlegen, deshalb ist das performant)


- das geht aber mit foldl nicht, weil ich da alles in den akkumulator reinschreiben muss vorher
- mit foldr allerdings schon, da ist der akku ja leer am anfang

`double(double 3)`
LO | LI
|--|--|
double 3 + double 3 | double (3+3)
(3+3) + double 3    | double 6
6 + double 3        | 6 + 6
6 + (3+3)           | 12
6+6                 | -

In [1]:
primes :: [Int]
primes = sieve [2..]
-- primes kann ich auch ausgeben, das ist eine unendliche liste ab 2

sieve :: [Int] -> [Int]
sieve (p:xs) = p : sieve (filter (\x -> x `mod` p /= 0) xs)


ones:: [Int]
ones = 1 : ones

-- ones !! 100000000000

primes !! 10

31